In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("NLP").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF

In [4]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish java could use classes"),
    (1.0, "Logistic,regression,models,are,neat")
], ["label", "sentence"])

In [5]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish java could...|
|  1.0|Logistic,regressi...|
+-----+--------------------+



In [6]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [7]:

words_data = tokenizer.transform(sentenceData)
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish java could...|[i, wish, java, c...|
|  1.0|Logistic,regressi...|[logistic,regress...|
+-----+--------------------+--------------------+



In [8]:
hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures")

In [9]:
featurized_data = hashing_tf.transform(words_data)

In [10]:
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [11]:
idf_model = idf.fit(featurized_data)

In [13]:
rescaled_data = idf_model.transform(featurized_data)
rescaled_data.select("label", "features").show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                            |
+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                         |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0  |(262144,[11534],[0.6931471805599453])                                                                                           

In [14]:
from pyspark.ml.feature import CountVectorizer

In [15]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [17]:
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

In [18]:
model = cv.fit(df)

In [19]:
result = model.transform(df)

In [21]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

